In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Projects\\Artificial Intelligence\\Kidney-Disease-Classification'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/codelikeabhi/Kidney-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "codelikeabhi"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "replace_with_your_tracking_password"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from KidneyDiseaseClassifier.constants import *
from KidneyDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:

        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/Dataset',
            mlflow_uri= 'https://dagshub.com/codelikeabhi/Kidney-Disease-Classification.mlflow',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
        )

        return eval_config

In [10]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:

    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation (self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_Score()

    def save_Score(self):
        scores = {"Loss" : self.score[0], "Accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)


    def log_into_mlflow(self):

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"Loss" : self.score[0], "Accuracy" : self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name = "VGG16Model")

            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-05-20 01:49:25,408: INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-05-20 01:49:25,446: INFO : common: yaml file: params.yaml loaded successfully]
[2024-05-20 01:49:25,459: INFO : common: Created Directory at artifacts]
Found 80 images belonging to 2 classes.
5/5 [==============================] - 13s 2s/step - loss: 6.6120 - accuracy: 0.5000
[2024-05-20 01:49:40,931: INFO : common: JSON file saved at: scores.json]
[2024-05-20 01:49:41,426: WARNING : connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /codelikeabhi/Kidney-Disease-Classification.mlflow/api/2.0/mlflow/runs/create]


2024/05/20 01:49:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-20 01:49:49,403: WARNING : save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\abhis\AppData\Local\Temp\tmpsbh58y0e\model\data\model\assets
[2024-05-20 01:49:51,213: INFO : builder_impl: Assets written to: C:\Users\abhis\AppData\Local\Temp\tmpsbh58y0e\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/20 01:51:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
